In [1]:
import transformers

import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print("Invalid device or cannot modify virtual devices once initialized.")
    pass

In [3]:
tf.__version__

'2.3.0'

In [4]:
train_df = pd.read_json('data/train.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [5]:
val_df = pd.read_json('data/dev.jsonl', lines = True)
val_df["img"] = "data/" + val_df["img"]

In [6]:
img_width = 224
img_height = 224

In [7]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [8]:
max_len = 128

### Train

In [9]:
def build_model():
    
    encoder = transformers.TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    encoder.trainable = False

    input_img = tf.keras.layers.Input(
        shape = (img_width, img_height, 3), name = "image"
    )
    
    input_ids = tf.keras.layers.Input(name = "input_ids", shape = (max_len,), dtype = tf.int32)

    attention_mask = tf.keras.layers.Input(name = "attention_mask", shape = (max_len,), dtype = tf.int32)

    y = encoder(input_ids, attention_mask = attention_mask)[0]
    y = tf.keras.layers.BatchNormalization()(y)
    
    y = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences = True))(y)

    extractor = tf.keras.applications.EfficientNetB7(include_top = False, \
                                                 input_tensor = input_img, weights = "imagenet")
    
    extractor.trainable = False
    
    for layer in extractor.layers:
        if (layer.name.startswith("block7") or layer.name.startswith("block6")):
            layer.trainable = True
    
    x = tf.keras.layers.GlobalMaxPooling2D()(extractor.output)
    
    y = tf.keras.layers.GlobalMaxPooling1D()(y)
    
    x = tf.keras.layers.concatenate([x, y])
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(1024, activation = "relu")(x)
    
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = tf.keras.models.Model([input_img, input_ids, attention_mask], out)

    return model

In [10]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = build_model()
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

block5h_project_conv (Conv2D)   (None, 14, 14, 224)  301056      block5h_se_excite[0][0]          
__________________________________________________________________________________________________
block5h_project_bn (BatchNormal (None, 14, 14, 224)  896         block5h_project_conv[0][0]       
__________________________________________________________________________________________________
block5h_drop (Dropout)          (None, 14, 14, 224)  0           block5h_project_bn[0][0]         
__________________________________________________________________________________________________
block5h_add (Add)               (None, 14, 14, 224)  0           block5h_drop[0][0]               
                                                                 block5g_add[0][0]                
__________________________________________________________________________________________________
block5i_expand_conv (Conv2D)    (None, 14, 14, 1344) 301056      block5h_add[0][0]                
__________

In [11]:
def encode_single_sample(img_path, label, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf')
    return {"image": img, "label": label, "text": text}

In [12]:
class HatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, label, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text
        self.label = label

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_label = self.label[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        z = np.zeros((self.batch_size), dtype = "float32")
        
        for i,j,k,l in zip(range(self.batch_size), batch_input_img_paths, batch_label, batch_text):
            sample = encode_single_sample(j,k,l)
            w[i] = sample["image"].numpy().tolist()
            input_ids = sample["text"]['input_ids'].numpy().tolist()[0]
            x[i] = input_ids + [0]*(max_len - len(input_ids))
            attention_mask = sample["text"]['attention_mask'].numpy().tolist()[0]
            y[i] = attention_mask + [0]*(max_len - len(attention_mask))
            z[i] = sample["label"]
        
        return [w,x,y], z

In [13]:
train_gen = HatefulMemes(16, train_df["img"].values.tolist(), train_df["label"].values.tolist(), \
                         train_df["text"].values.tolist())
val_gen = HatefulMemes(16, val_df["img"].values.tolist(), val_df["label"].values.tolist(), \
                         val_df["text"].values.tolist())

In [14]:
epochs = 50
early_stopping_patience = 10

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_accuracy", patience = early_stopping_patience, restore_best_weights = True
)

history = model.fit(train_gen, validation_data = val_gen, epochs = epochs, callbacks = [early_stopping])

Epoch 1/50
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
531/531 [==============================] - 1539s 3s/step - loss: 0.6913 - accuracy: 0.6106 - val_loss: 1.0290 - val_accuracy: 0.4859
Epoch 2/50
531/531 [==============================] - 1400s 3s/step - loss: 0.6168 - accuracy: 0.6704 - val_loss: 0.7410 - val_accuracy: 0.5605
Epoch 3/50
531/531 [==============================] - 1426s 3s/step - loss: 0.5962 - accuracy: 0.6874 - val_loss: 0.7323 - val_accuracy: 0.5544
Epoch 4/50
531/531 [==============================] - 1451s 3s/step - loss: 0.5733 - accuracy: 0.7007 - val_loss: 0.7787 - val_accuracy: 0.5383
Epoch 5/50
531/531 [==============================] - 1456s 3s/step - loss: 0.5554 - accuracy: 0.7157 - val_loss: 0.8531 - val_accuracy: 0.5161
Epoch 6/50
531/531 [==============================] - 1481s 3s/step - loss: 0.5412 - accuracy: 0.7269 - val_loss: 0.8590 - val_accuracy: 0.5262
Epoch 7/50
531/531 [==============================] - 

### Predict

In [15]:
train_df = pd.read_json('data/test.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [16]:
def val_encode_single_sample(img_path, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf')
    return {"image": img, "text": text}

In [17]:
class ValHatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        
        for i,j,k in zip(range(self.batch_size), batch_input_img_paths, batch_text):
            sample = val_encode_single_sample(j,k)
            w[i] = sample["image"].numpy().tolist()
            input_ids = sample["text"]['input_ids'].numpy().tolist()[0]
            for _ in range(max_len - len(input_ids)):
              input_ids.append(0)
            x[i] = input_ids
            attention_mask = sample["text"]['attention_mask'].numpy().tolist()[0]
            for _ in range(max_len - len(attention_mask)):
              attention_mask.append(0)
            y[i] = attention_mask
        
        return [w,x,y]

In [18]:
pred_gen = ValHatefulMemes(1, train_df["img"].values.tolist(), train_df["text"].values.tolist())

In [19]:
preds = model.predict(pred_gen, verbose = 1)

1000/1000 [==============================] - 221s 221ms/step


In [20]:
def prob2pred(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [21]:
train_df["proba"] = [i[0] for i in preds.tolist()]
train_df["label"] = train_df["proba"].apply(prob2pred)

In [22]:
train_df = train_df.drop(["img","text"], axis = 1)
train_df

,id,proba,label
0,16395,0.475429,0
1,37405,0.297154,0
2,94180,0.476164,0
3,54321,0.339373,0
4,97015,0.668694,1
...,...,...,...
995,3869,0.256348,0
996,23817,0.395303,0
997,56280,0.277066,0
998,29384,0.120962,0


In [23]:
train_df.to_csv("submission_new.csv", index = False)